## Libraries

In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt
import time

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

## Hyperparameters

In [2]:
ENV = 'LunarLander-v2'

EPISODES = 50000 # Number of times the enviroment is ran

LOSS_CLIPPING = 0.2 # Approximated values stated in the original paper
EPOCHS = 20 # Epochs to train the network (recommended between 3 and 30)
NOISE = 1.0 # Standard deviation

GAMMA = 0.99 # Used for the estimated reward

BUFFER_SIZE = 1024 # Buffer of experiences
BATCH_SIZE = 512 # Batch size for the neural nets
NUM_ACTIONS = 4 # Number of possible action in the environment
NUM_STATES = 8 # Number of possible states in the exvironment
ENTROPY_LOSS = 0.001 # Constant value for the entropy (entropy used for exploration)
LR = 0.00025  # Learning rate

# Used to feed the actor when predicting
DUMMY_ACTION, DUMMY_VALUE = np.zeros((1, NUM_ACTIONS)), np.zeros((1, 1))

## Class memory


In [3]:
# Memory of the experiences
class Memory:
    def __init__(self):
        self.episode_batch = [[], [], []]
        self.global_batch = [[], [], [], []]
        self.episode_rewards = []
        self.historical_rewards = []

    def reset_espisode_batch(self):
        self.global_batch = [[], [], [], []]
        self.episode_batch = [[], [], []]
        self.episode_rewards = []

## Custom loss function for PPO

In [4]:
# Custom loss functions for the PPO
def proximal_policy_optimization_loss(advantages, prediction_picks):
    def loss(actions, y_pred):
        #advantages, prediction_picks, actions = y_true[:, :1], y_true[:, 1:1+self.action_space], y_true[:, 1+self.action_space:]
        LOSS_CLIPPING = 0.2
        ENTROPY_LOSS = 0.001
        
        prob = actions * y_pred
        old_prob = actions * prediction_picks

        prob = K.clip(prob, 1e-10, 1.0)
        old_prob = K.clip(old_prob, 1e-10, 1.0)

        ratio = K.exp(K.log(prob) - K.log(old_prob))
        
        p1 = ratio * advantages
        p2 = K.clip(ratio, min_value=1 - LOSS_CLIPPING, max_value=1 + LOSS_CLIPPING) * advantages

        actor_loss = -K.mean(K.minimum(p1, p2))

        entropy = -(y_pred * K.log(y_pred + 1e-10))
        entropy = ENTROPY_LOSS * K.mean(entropy)
        
        total_loss = actor_loss - entropy

        return total_loss
    return loss

## PPO Agent

In [5]:
class Agent:
    # Constructor of the class
    def __init__(self):
        self.actor = self.create_actor()
        self.critic = self.create_critic()
        self.memory = Memory()

    # Create the actor used to select the action given an state
    def create_actor(self):
        # Define three inputs as the advantage and old prediction is used for the custom loss
        input_state = Input(shape=(NUM_STATES,))
        input_advantage = Input(shape=(1,))
        imput_old_prediction = Input(shape=(NUM_ACTIONS,))

        x = Dense(512, activation='relu', kernel_initializer=tf.random_normal_initializer(stddev=0.01))(input_state)
        x = Dense(256, activation='relu', kernel_initializer=tf.random_normal_initializer(stddev=0.01))(x)
        x = Dense(64, activation='relu', kernel_initializer=tf.random_normal_initializer(stddev=0.01))(x)

        # Softmax as there are different probabilities depending on the action
        output_layer = Dense(NUM_ACTIONS, activation='softmax', name='output')(x)

        model = Model(inputs=[input_state, input_advantage, imput_old_prediction], outputs=[output_layer])

        # Compile the model with the custom loss
        model.compile(optimizer=Adam(lr=LR),
                      loss=[proximal_policy_optimization_loss(
                          advantages=input_advantage,
                          prediction_picks=imput_old_prediction)])
        model.summary()

        return model

    # Create the critic which will criticise how the actor is performing
    def create_critic(self):
        # Define the architectire of the network
        input_layer = Input(shape=(NUM_STATES,))
        
        x = Dense(512, activation='relu', kernel_initializer=tf.random_normal_initializer(stddev=0.01))(input_layer)
        x = Dense(256, activation='relu', kernel_initializer=tf.random_normal_initializer(stddev=0.01))(x)
        x = Dense(64, activation='relu', kernel_initializer=tf.random_normal_initializer(stddev=0.01))(x)


        # Linear output to know how good the action is
        ouput_layer = Dense(1)(x)

        model = Model(inputs=[input_layer], outputs=[ouput_layer])

        # Compile it with mse loss and gradient descent
        model.compile(optimizer=Adam(lr=LR), 
                      loss='mse')

        return model

    # Get the action given the current state
    def get_action(self, state):
        probabilities = self.actor.predict([state.reshape(1, NUM_STATES), DUMMY_VALUE, DUMMY_ACTION])
        action = np.random.choice(NUM_ACTIONS, p=np.nan_to_num(probabilities[0]))
        action_matrix = np.zeros(NUM_ACTIONS)
        action_matrix[action] = 1
        return action, action_matrix, probabilities

    # Transform rewards of the episode as the discount has to be applied in the first ones regarding the last ones
    def transform_reward(self):
        self.memory.historical_rewards.append(sum(self.memory.episode_rewards))

        for j in range(len(self.memory.episode_rewards) - 2, -1, -1):
            self.memory.episode_rewards[j] += self.memory.episode_rewards[j + 1] * GAMMA

## Environment

In [6]:
# Create simulated environment
def create_environment():
    environment = gym.make(ENV)
    return environment

In [7]:
if __name__ == '__main__':
    start = time.time()
    env = create_environment() 
    ag = Agent()
    episode = 1
    goal_reached = False

    # Iterate until the number of episodes is reached
    while episode < EPISODES:
        # Reset episode and global batch and the environment to start a new one
        ag.memory.reset_espisode_batch()
        state = env.reset()
        states = []
        next_states = []
        dones = []
        # Iterate until the global batch is bigger than the buffer
        while len(ag.memory.global_batch[0]) < BUFFER_SIZE:
            # Get the action given the current state
            action, action_matrix, predicted_action = ag.get_action(state)
            # Step in the enviroment and gather all parameters
            next_state, reward, done, info = env.step(action)
            ag.memory.episode_rewards.append(reward)
            last_reward = sum(ag.memory.episode_rewards)

            # Add to the episode batch the informetions gathered
            ag.memory.episode_batch[0].append(state)
            ag.memory.episode_batch[1].append(action_matrix)
            ag.memory.episode_batch[2].append(predicted_action)
            states.append(state)
            next_states.append(next_state)
            state = next_state
            dones.append(done)

            # If the episode has finished
            if done:
                # Transform rewards with discount rate
                ag.transform_reward()
                print('Last reward: ', ag.memory.episode_rewards[-1])
                # Iterate through all instances of the episode batch
                for i in range(len(ag.memory.episode_batch[0])):
                    obs, action, pred = ag.memory.episode_batch[0][i], ag.memory.episode_batch[1][i], ag.memory.episode_batch[2][i]
                    r = ag.memory.episode_rewards[i]

                    # Add all experienes to global batch
                    ag.memory.global_batch[0].append(obs)
                    ag.memory.global_batch[1].append(action)
                    ag.memory.global_batch[2].append(pred)
                    ag.memory.global_batch[3].append(r)
                
                # Reset the episode batch, episode reward and environment
                ag.memory.episode_batch = [[], [], []]
                ag.memory.episode_rewards = []
                state = env.reset()

                if episode >= 100:
                    print('EPISODE: ', episode, ' - SCORE: ', ag.memory.historical_rewards[-1], ' - MEAN SCORE: ', sum(ag.memory.historical_rewards[-100:])/100)

                    if sum(ag.memory.historical_rewards[-100:])/100 >= 200:
                        goal_reached = True
                        break
                else:
                    print('EPISODE: ', episode, ' - SCORE: ', ag.memory.historical_rewards[-1], ' - MEAN SCORE: ', sum(ag.memory.historical_rewards)/len(ag.memory.historical_rewards))
                
                episode += 1

        if goal_reached == True:
            break

        # Get the arrays to feed te neural nets
        obs, action, pred, reward = np.array(ag.memory.global_batch[0]), np.array(ag.memory.global_batch[1]), np.array(ag.memory.global_batch[2]), np.reshape(np.array(ag.memory.global_batch[3]), (len(ag.memory.global_batch[3]), 1))
        pred = np.reshape(pred, (pred.shape[0], pred.shape[2]))

        # Ensure that they all have the same size
        obs, action, pred, reward = obs[:BUFFER_SIZE], action[:BUFFER_SIZE], pred[:BUFFER_SIZE], reward[:BUFFER_SIZE]
        states, next_states = np.array(states[:BUFFER_SIZE]), np.array(next_states[:BUFFER_SIZE])
        old_prediction = pred

        
        
        
        values = ag.critic.predict(states)
        next_values = ag.critic.predict(next_states)
        gamma = 0.99
        lamda = 0.9
        deltas = [r + gamma * (1 - d) * nv - v for r, d, nv, v in zip(reward, dones, next_values, values)]
        
        for t in reversed(range(len(deltas) - 1)):
            deltas[t] = deltas[t] + (1 - dones[t]) * gamma * lamda * deltas[t + 1]
        #print(np.shape(deltas))
        deltas = np.array(deltas)
        
        deltas = (deltas - deltas.mean()) / (deltas.std() + 1e-8)
        
        
        # Get advantage with the rewards taken from the environment predictions of the critic given the states
        #advantage = reward - pred_values
        print('ACTUALIZA')
        # Update weights of the actor with the states, advanges obtained from the critic and predictions made. Target values are actions
        actor_loss = ag.actor.fit([obs, deltas, old_prediction], [action], batch_size=BATCH_SIZE, shuffle=True, epochs=EPOCHS, verbose=0)
        # Update weights of the critic with the states. Targe values are actual rewards
        critic_loss = ag.critic.fit([obs], [reward], batch_size=BATCH_SIZE, shuffle=True, epochs=EPOCHS, verbose=0)

    # Plot a graph with the rewards over the episodes
    plt.plot(ag.memory.historical_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Score")
    
    end = time.time()
    print((end-start)/60, 'MINUTES')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          4608        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          131328      dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           16448       dense_1[0][0]                    
_____________

Last reward:  -100
EPISODE:  62  - SCORE:  -244.83831116307974  - MEAN SCORE:  -151.33635020312946
Last reward:  -100
EPISODE:  63  - SCORE:  -93.56846569654215  - MEAN SCORE:  -150.41939965540584
Last reward:  -100
EPISODE:  64  - SCORE:  -110.493256088692  - MEAN SCORE:  -149.79555366217593
Last reward:  -100
EPISODE:  65  - SCORE:  -136.15373792059026  - MEAN SCORE:  -149.58567957384386
Last reward:  -100
EPISODE:  66  - SCORE:  -254.02221814685174  - MEAN SCORE:  -151.16805137040458
Last reward:  -100
EPISODE:  67  - SCORE:  -161.67726496291357  - MEAN SCORE:  -151.32490530462113
Last reward:  -100
EPISODE:  68  - SCORE:  -124.75863661113293  - MEAN SCORE:  -150.9342248826581
Last reward:  -100
EPISODE:  69  - SCORE:  -135.95922057150355  - MEAN SCORE:  -150.71719583467035
Last reward:  -100
EPISODE:  70  - SCORE:  -134.82254408288264  - MEAN SCORE:  -150.4901293810734
Last reward:  -100
EPISODE:  71  - SCORE:  -209.06617676520614  - MEAN SCORE:  -151.31514413296256
Last reward:  -

Last reward:  -100
EPISODE:  145  - SCORE:  -250.6326173010915  - MEAN SCORE:  -158.7886545351671
Last reward:  -100
EPISODE:  146  - SCORE:  -112.90293888865398  - MEAN SCORE:  -158.7956017572812
Last reward:  -100
EPISODE:  147  - SCORE:  -114.69385603229117  - MEAN SCORE:  -157.6999980635805
Last reward:  -100
EPISODE:  148  - SCORE:  -193.79165466356534  - MEAN SCORE:  -158.61170015416562
Last reward:  -100
EPISODE:  149  - SCORE:  -189.0107400082078  - MEAN SCORE:  -159.32645869020504
Last reward:  -100
EPISODE:  150  - SCORE:  -158.69560443526592  - MEAN SCORE:  -159.61221393033065
ACTUALIZA
Last reward:  -100
EPISODE:  151  - SCORE:  -103.7049517338339  - MEAN SCORE:  -160.95220770313372
Last reward:  -100
EPISODE:  152  - SCORE:  -161.9045966933516  - MEAN SCORE:  -161.52177648489157
Last reward:  -100
EPISODE:  153  - SCORE:  -216.56246962801762  - MEAN SCORE:  -162.31967207186307
Last reward:  -100
EPISODE:  154  - SCORE:  -165.0332501901438  - MEAN SCORE:  -161.8738815298521

Last reward:  -100
EPISODE:  227  - SCORE:  -141.07405104260684  - MEAN SCORE:  -138.54900776700472
Last reward:  -100
EPISODE:  228  - SCORE:  -177.2278068106695  - MEAN SCORE:  -138.23567333375112
Last reward:  -100
EPISODE:  229  - SCORE:  -107.46443056291423  - MEAN SCORE:  -138.1400024986598
Last reward:  -100
EPISODE:  230  - SCORE:  -201.85925104469186  - MEAN SCORE:  -139.33420526517176
Last reward:  -100
EPISODE:  231  - SCORE:  -79.60076552525611  - MEAN SCORE:  -138.76464800936049
Last reward:  -100
EPISODE:  232  - SCORE:  -74.04169082368523  - MEAN SCORE:  -138.35004723149382
Last reward:  -100
EPISODE:  233  - SCORE:  -145.66062424295944  - MEAN SCORE:  -137.94544490597266
Last reward:  -100
EPISODE:  234  - SCORE:  -107.95475760867842  - MEAN SCORE:  -136.20340078849418
ACTUALIZA
Last reward:  -100
EPISODE:  235  - SCORE:  -165.83593919025625  - MEAN SCORE:  -134.6772904244385
Last reward:  -100
EPISODE:  236  - SCORE:  -63.794768830566284  - MEAN SCORE:  -133.5701617546

Last reward:  -100
EPISODE:  310  - SCORE:  -117.50641187086761  - MEAN SCORE:  -128.19531921502187
Last reward:  -100
EPISODE:  311  - SCORE:  -86.5698356581359  - MEAN SCORE:  -128.10936814494164
Last reward:  -100
EPISODE:  312  - SCORE:  -132.9592213466031  - MEAN SCORE:  -128.13862033621785
ACTUALIZA
Last reward:  -100
EPISODE:  313  - SCORE:  -85.65615166612919  - MEAN SCORE:  -128.10061131122922
Last reward:  -100
EPISODE:  314  - SCORE:  -106.2720301367716  - MEAN SCORE:  -128.03678491010177
Last reward:  -100
EPISODE:  315  - SCORE:  -135.38381743403363  - MEAN SCORE:  -127.89506773465686
Last reward:  -100
EPISODE:  316  - SCORE:  -134.8442892656309  - MEAN SCORE:  -128.15869516117314
Last reward:  -100
EPISODE:  317  - SCORE:  -131.4941074938049  - MEAN SCORE:  -127.56501807701305
Last reward:  -100
EPISODE:  318  - SCORE:  -110.34526313535244  - MEAN SCORE:  -127.74966162744948
Last reward:  -100
EPISODE:  319  - SCORE:  -97.6153952236985  - MEAN SCORE:  -127.97992569037831

...

Last reward:  -100
EPISODE:  2358  - SCORE:  -118.99008616306284  - MEAN SCORE:  -122.3407513580467
Last reward:  -100
EPISODE:  2359  - SCORE:  -116.85596958106657  - MEAN SCORE:  -122.31308934424996
Last reward:  -100
EPISODE:  2360  - SCORE:  -108.8231787349056  - MEAN SCORE:  -122.48611898713935
Last reward:  -100
EPISODE:  2361  - SCORE:  -103.93259063614873  - MEAN SCORE:  -122.45659495959067
Last reward:  -100
EPISODE:  2362  - SCORE:  -108.83992778584731  - MEAN SCORE:  -122.16112905941777
Last reward:  -100
EPISODE:  2363  - SCORE:  -126.29300582942803  - MEAN SCORE:  -122.13554910677767
Last reward:  -100
EPISODE:  2364  - SCORE:  -93.42928242309952  - MEAN SCORE:  -121.49016488072363
Last reward:  -100
EPISODE:  2365  - SCORE:  -134.11223309956475  - MEAN SCORE:  -121.78519092426052
ACTUALIZA
Last reward:  -100
EPISODE:  2366  - SCORE:  -100.1041537988574  - MEAN SCORE:  -121.58912985517901
Last reward:  -100
EPISODE:  2367  - SCORE:  -100.24583115332996  - MEAN SCORE:  -121

Last reward:  -100
EPISODE:  2440  - SCORE:  -157.57976478325784  - MEAN SCORE:  -121.82315320953188
Last reward:  -100
EPISODE:  2441  - SCORE:  -90.23488832479731  - MEAN SCORE:  -121.47034995766485
Last reward:  -100
EPISODE:  2442  - SCORE:  -136.27969411478335  - MEAN SCORE:  -121.30232678952892
Last reward:  -100
EPISODE:  2443  - SCORE:  -122.50938069656821  - MEAN SCORE:  -121.45705576307687
Last reward:  -100
EPISODE:  2444  - SCORE:  -105.69465211856573  - MEAN SCORE:  -121.19456698246813
Last reward:  -100
EPISODE:  2445  - SCORE:  -135.7116387945718  - MEAN SCORE:  -121.27597745367775
Last reward:  -100
EPISODE:  2446  - SCORE:  -116.29961745282117  - MEAN SCORE:  -121.14931056533987
Last reward:  -100
EPISODE:  2447  - SCORE:  -115.69110090852377  - MEAN SCORE:  -121.33345150833921
Last reward:  -100
EPISODE:  2448  - SCORE:  -83.08103379321628  - MEAN SCORE:  -120.8840022858048
Last reward:  -100
EPISODE:  2449  - SCORE:  -138.79155172436424  - MEAN SCORE:  -121.055910270

Last reward:  -100
EPISODE:  2523  - SCORE:  -120.01257817081168  - MEAN SCORE:  -125.80247888615676
Last reward:  -100
EPISODE:  2524  - SCORE:  -128.52007815380432  - MEAN SCORE:  -125.86695417130008
Last reward:  -100
EPISODE:  2525  - SCORE:  -95.10293618833998  - MEAN SCORE:  -125.6210028230217
Last reward:  -100
EPISODE:  2526  - SCORE:  -108.31239924570448  - MEAN SCORE:  -125.23673896651322
Last reward:  -100
EPISODE:  2527  - SCORE:  -150.06052270516176  - MEAN SCORE:  -125.45996992937476
Last reward:  -100
EPISODE:  2528  - SCORE:  -123.4094478811036  - MEAN SCORE:  -125.40911507909637
Last reward:  -100
EPISODE:  2529  - SCORE:  -138.26240065506303  - MEAN SCORE:  -125.24880450698484
ACTUALIZA
Last reward:  -100
EPISODE:  2530  - SCORE:  -142.31508619263712  - MEAN SCORE:  -125.51704862553468
Last reward:  -100
EPISODE:  2531  - SCORE:  13.513706290491456  - MEAN SCORE:  -123.98264377854561
Last reward:  -100
EPISODE:  2532  - SCORE:  -128.8150531004909  - MEAN SCORE:  -124.

Last reward:  -100
EPISODE:  2605  - SCORE:  -115.58976741890807  - MEAN SCORE:  -127.98307618430498
Last reward:  -100
EPISODE:  2606  - SCORE:  -143.09082690275122  - MEAN SCORE:  -127.99633901336387
Last reward:  -100
EPISODE:  2607  - SCORE:  -183.09385119185833  - MEAN SCORE:  -127.94134754728475
Last reward:  -100
EPISODE:  2608  - SCORE:  -118.54812025384577  - MEAN SCORE:  -127.54065624958874
Last reward:  -100
EPISODE:  2609  - SCORE:  -130.85777454646424  - MEAN SCORE:  -127.14249646538156
Last reward:  -100
EPISODE:  2610  - SCORE:  -125.63412456738135  - MEAN SCORE:  -126.94561091861969
Last reward:  -100
EPISODE:  2611  - SCORE:  -117.9377807113748  - MEAN SCORE:  -126.73887794955512
Last reward:  -100
EPISODE:  2612  - SCORE:  -120.21163161532432  - MEAN SCORE:  -126.67122540269902
Last reward:  -100
EPISODE:  2613  - SCORE:  -117.03946017054282  - MEAN SCORE:  -126.71002558286658
Last reward:  -100
EPISODE:  2614  - SCORE:  -183.05886767594728  - MEAN SCORE:  -127.324425

KeyboardInterrupt: 